In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import *

In [2]:
path_dir_update = "/home/ale1726/proyects/datalake/clientes/data/ETL/24_02_2025"
df = pd.read_csv(os.path.join(path_dir_update,"CLientesT_FINAL_ACTIVOS.csv"))

In [126]:
df.describe()

,ID_DL,ENTIDAD_FEDERATIVA_NACIMIENTO,NUMERO_INTERIOR,COLONIA_URBANIZACION,FIEL,PRODUCTO_CONTRATADO
count,6868.000000,0.0,0.0,0.0,1.100000e+01,0.0
mean,3434.500000,NaN,NaN,NaN,1.000001e+15,NaN
std,1982.765157,NaN,NaN,NaN,1.505044e+08,NaN
min,1.000000,NaN,NaN,NaN,1.000000e+15,NaN
25%,1717.750000,NaN,NaN,NaN,1.000000e+15,NaN
50%,3434.500000,NaN,NaN,NaN,1.000001e+15,NaN
75%,5151.250000,NaN,NaN,NaN,1.000001e+15,NaN
max,6868.000000,NaN,NaN,NaN,1.000001e+15,NaN


In [118]:
df.value_counts("GENERO")

GENERO
MASCULINO    1633
NO_APLICA    1391
FEMENINO     1285
Name: count, dtype: int64

In [119]:
1285+1633

2918

In [ ]:
df["GENERO"] = df["GENERO"].apply(lambda x: x if x is not pd.NA else "NO_APLICA")

In [112]:
total_nulos = df.isnull().sum().sum()
total_valores = df.size
porcentaje_nulos = (total_nulos / total_valores) * 100

In [ ]:
print(f"Porcentaje total de valores nulos: {porcentaje_nulos:.2f}%")

In [128]:
registros_unicos = df.nunique(axis=0) 

In [129]:
registros_unicos

ID_DL                            6868
NEGOCIO                             2
NOMBRE_O_RAZON_SOCIAL            6867
NUMERO_CLIENTE                   6851
ESTATUS                             1
NUMERO_CONTRATO                  4309
GENERO                              3
FECHA_NAC_O_CONST                2647
ENTIDAD_FEDERATIVA_NACIMIENTO       0
PAIS_DE_NACIMIENTO                  4
NACIONALIDAD                       41
PROFESION                          33
CALLE                             179
NUMERO_EXTERIOR                    14
NUMERO_INTERIOR                     0
COLONIA_URBANIZACION                0
DELEGACION_MUNICIPIO               29
CIUDAD_POBLACION                   33
ENTIDAD_FEDERATIVA                 48
CODIGO_POSTAL                      13
PAIS                               32
TELEFONO                          667
CORREO_ELECTRONICO                 22
RFC                              5297
CURP                              137
FIEL                               11
REPRESENTANT

In [130]:
total_registros = len(df)
registros_unicos = df.nunique(axis=0)  # Cuenta valores únicos por columna

# Opcional: Si tienes una columna clave como "ID_Cliente", puedes hacer:
columna_clave = "RFC"  # Reemplaza con la columna relevante
unicidad = (df[columna_clave].nunique() / total_registros) * 100

print(f"Porcentaje de valores únicos en '{columna_clave}': {unicidad:.2f}%")

Porcentaje de valores únicos en 'RFC': 77.13%


In [122]:
total_registros = len(df)
duplicados = df.duplicated().sum()
porcentaje_duplicados = (duplicados / total_registros) * 100
print(f"Porcentaje de registros duplicados: {porcentaje_duplicados:.2f}%")


Porcentaje de registros duplicados: 0.00%


In [123]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
6863    False
6864    False
6865    False
6866    False
6867    False
Length: 6868, dtype: bool

In [107]:
df.value_counts("GENERO").sum()

4309

In [132]:
df["NOMBRE_O_RAZON_SOCIAL"].str.replace(r'[^\x00-\x7F]+', 'a', regex=True)

0        NAFIN FIDUCIARIO FIDEICOMISO CONTRAGARANTIA F...
1                  .+ TASA ACTIVO, S.A. DE C.V., F.I.R.V.
2              .+TASA LARGO PLAZO, S.A. DE C.V., F.I.I.D.
3              .+TASA PREVISIONAL, S.A. DE C.V., F.I.I.D.
4                       .+TASA US, S.A. DE C.V., F.I.I.D.
                              ...                        
6863    ZURICH SANTANDER SEGUROS MEXICO, S.A. /1365/04...
6864    ZURICH SANTANDER SEGUROS MEXICO, S.A. /DESIN 1...
6865                ZURICH SANTANDER SEGUROS MaXICO, S.A.
6866                ZURICH VIDA COMPAaIA DE SEGUROS, S.A.
6867                                 primas de antiguedad
Name: NOMBRE_O_RAZON_SOCIAL, Length: 6868, dtype: object

In [ ]:
df["NOMBRE_O_RAZON_SOCIAL"] = df["NOMBRE_O_RAZON_SOCIAL"].str.replace(r'[^\x00-\x7F]+', 'a', regex=True)


In [4]:
df_copy = df.copy()

In [5]:
df_copy["S2"] = df_copy["SISTEMA_ORIGEN"].apply(lambda x: x if len(x.split(',')) <= 1 else x.split(',')[0])

In [88]:
len("SIRAC".split(','))

1

In [7]:
df_copy[df_copy["TIPO_PERSONA"] != "PF"].value_counts("S2")

S2
TAS      1958
SIRAC    1389
MECA      208
SIPE      178
SIAG      111
SIMS       84
SOI        22
Name: count, dtype: int64

In [103]:
[
    {'name': sistema, 'y': value}
    for sistema, value in df_copy.value_counts("S2").to_dict().items()
]

[{'name': 'SIRAC', 'y': 4307},
 {'name': 'TAS', 'y': 1958},
 {'name': 'MECA', 'y': 208},
 {'name': 'SIPE', 'y': 178},
 {'name': 'SIAG', 'y': 111},
 {'name': 'SIMS', 'y': 84},
 {'name': 'SOI', 'y': 22}]

dict_items([('SIRAC', 4307), ('TAS', 1958), ('MECA', 208), ('SIPE', 178), ('SIAG', 111), ('SIMS', 84), ('SOI', 22)])

In [85]:
"SIRAC, TAS".split(',')[0]

'SIRAC'

In [80]:
df.value_counts("SISTEMA_ORIGEN")

SISTEMA_ORIGEN
SIRAC                           4278
TAS                             1934
MECA                             208
SIPE                             152
SIAG                             107
SIMS                              77
SOI                               22
SIRAC, TAS                        22
TAS, MECA                         18
SIPE, MECA                         9
SIPE, TAS                          7
SIPE, MECA, TAS                    5
SIRAC, SIAG                        4
SIMS, SIPE                         3
SIAG, MECA                         3
SIMS, TAS                          2
TAS, SIAG                          2
TAS, SIMS, MECA                    2
SIRAC, TAS, SIAG                   2
TAS, SIMS, MECA, SIAG              1
SIPE, SIRAC                        1
SIRAC, MECA, TAS                   1
SIPE, TAS, SIAG                    1
SIPE, SIMS, MECA                   1
SIPE, MECA, TAS, SIAG              1
SIPE, MECA, SIAG                   1
SIMS, SIAG             

In [75]:
print(df[df["NOMBRE_O_RAZON_SOCIAL"].apply(lambda x: "�" in x if isinstance(x, str) else False)].to_string(max_cols=None))


      ID_DL NEGOCIO                                                    NOMBRE_O_RAZON_SOCIAL                               NUMERO_CLIENTE  ESTATUS NUMERO_CONTRATO GENERO FECHA_NAC_O_CONST  ENTIDAD_FEDERATIVA_NACIMIENTO PAIS_DE_NACIMIENTO NACIONALIDAD                                PROFESION CALLE NUMERO_EXTERIOR  NUMERO_INTERIOR  COLONIA_URBANIZACION DELEGACION_MUNICIPIO CIUDAD_POBLACION ENTIDAD_FEDERATIVA CODIGO_POSTAL PAIS                               TELEFONO CORREO_ELECTRONICO                          RFC CURP  FIEL                                                                                                                                          REPRESENTANTE_LEGAL                TIPO_PERSONA  PRODUCTO_CONTRATADO SISTEMA_ORIGEN FECHA_DE_ACTUALIZACION
23       23     NaN                          ABC CAPITAL, S.A. INSTITUCI�N DE BANCA M�LTIPLE                                      1059995  VIGENTE             NaN    NaN               NaN                            NaN           

In [ ]:
df

In [ ]:
# Aplicar la corrección con la codificación detectada
df["NOMBRE_O_RAZON_SOCIAL"] = df["NOMBRE_O_RAZON_SOCIAL"].astype(str).apply(lambda x: x.encode(detected_encoding, errors="ignore").decode("utf-8") if isinstance(x, str) else x)


In [42]:
df.loc[df["PAIS"] == "MEXICO", "NOMBRE_O_RAZON_SOCIAL"].sample(10)


1826      ELECTROTECNIA PLUS SAPI DE CV
219               ALFREDO CABADA MORENO
1415    CONFECCIONES MODA PIEL SA DE CV
3967         MANUEL BARCELATA RODRIGUEZ
2622            GILBERTO ENRIQUEZ TREJO
3437           JOSE LUIS MARTINEZ NIETO
1583              DAVID SANTIAGO NOYOLA
3862          LUZ ELENA AVILA HERNANDEZ
4247                MARINA PEREZ TORRES
3407      JOSE GUADALUPE URRUTIA TORRES
Name: NOMBRE_O_RAZON_SOCIAL, dtype: object

In [44]:
[
    {"Cliente": str(fil.NOMBRE_O_RAZON_SOCIAL)}
    for fil in df.loc[df["PAIS"] == "MEXICO", ["NOMBRE_O_RAZON_SOCIAL"]].sample(10).itertuples(index=False)
]


[{'Cliente': 'OSWALDO CORONA AMADOR'},
 {'Cliente': 'JENNIFER IXTACCIHUATL CAMPOS VAZQUEZ'},
 {'Cliente': 'LUCINA MALDONADO VILLANUEVA'},
 {'Cliente': 'SANTIAGO IX TEC'},
 {'Cliente': 'PELICULAS Y EMPAQUES ESPECIALES S.A. DE C.V.'},
 {'Cliente': 'ENRIQUE JESUS LIMA MUÑOZ LEDO'},
 {'Cliente': 'JOSE ELIAS SAHAB JAIK'},
 {'Cliente': 'JOSE DE JESUS ROMO CABRERA'},
 {'Cliente': 'RECURSOS MEDICOS Y TECNOLOGICOS PARA LA SALUD SA'},
 {'Cliente': 'ARMANDO MELENDEZ GARCIA'}]

In [45]:
df[df["PAIS"] == "VENEZUELA"]

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
1468,1411,NaN,CORPORACION ANDINA DE FOMENTO (CAF),95,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,NaN,SOI,24/02/25


In [49]:
mapping_pais = {
    "MEX": "MEXICO",
    "U.S.A.": "ESTADOS UNIDOS DE AMERICA",
    "ESTADOS UNIDOS": "ESTADOS UNIDOS DE AMERICA",
    "GRAN BRETA&A": "REINO UNIDO",
    "GRAN BRETA\A":"REINO UNIDO",
    "ISLAS GRAND CAYMAN": "REINO UNIDO",
    "ESPA\A": "ESPAÑA",
    "ESPANA": "ESPAÑA",
    "ESPA\u00d1A":"ESPAÑA",
    "NASSAU": "BAHAMAS"
}


In [50]:
def clean_pais(pais):
    if pd.isna(pais):
        return pais
    if len(pais.split(',')) == 1:
        return mapping_pais.get(pais, pais)
    else:
        cadena_limpia = list(set([clean_pais(p.strip()) for p in pais.split(",")])) 
        return ", ".join(cadena_limpia)
    
df["PAIS"] =  df["PAIS"].apply(clean_pais)

In [60]:
[
    {"Cliente": str(fil.NOMBRE_O_RAZON_SOCIAL)}
    for fil in df.loc[(df["PAIS"] == "MEXICO") & (df["TIPO_PERSONA"] == "PF"), ["NOMBRE_O_RAZON_SOCIAL"]]
    .sample(15)
    .itertuples(index=False)
]


[{'Cliente': 'ARTURO MENDEZ DEL CASTILLO'},
 {'Cliente': 'SALVADOR DOMINGUEZ LOERA'},
 {'Cliente': 'JUAN ENRIQUE ALDANA ZERMENO'},
 {'Cliente': 'ANDRES ROJAS FLORES'},
 {'Cliente': 'PABLO JAHIR SANCHEZ RON'},
 {'Cliente': 'MA TRINIDAD CHACON GALAVIZ'},
 {'Cliente': 'RICARDO VALDEZ NAVA'},
 {'Cliente': 'ANGELES LIZETH SIBAJA ARROYO'},
 {'Cliente': 'PRIMITIVO ANGUIANO HERNANDEZ'},
 {'Cliente': 'JOSE DE JESUS DELGADO SANCHEZ'},
 {'Cliente': 'MAGDALENA CASTILLON PENA'},
 {'Cliente': 'JOSE LUIS OLIVO CASTILLO'},
 {'Cliente': 'ROGELIO MENDOZA SAUCEDO'},
 {'Cliente': 'CESAR PAUL GARCIA FLORES'},
 {'Cliente': 'ELSA PATRICIA SALAS AYALA'}]

In [16]:
col = "TIPO_PERSONA"
df.value_counts(col)

TIPO_PERSONA
PF                                                         2918
PM                                                         1834
Moral Nacional no Gravable                                 1571
Moral Nacional Gravable                                     314
UNIONES DE CREDITO                                          106
Moral Nacional no Gravable, PM                               43
Cuentas Propias                                              36
PM, Moral Nacional Gravable                                  13
UNIONES DE CREDITO, PM                                        9
Moral Nacional no Gravable, Moral Nacional Gravable           6
Moral Nacional no Gravable, PM, UNIONES DE CREDITO            6
Moral Nal.No Gravable (Fondos)                                4
Moral Nacional no Gravable, UNIONES DE CREDITO                2
BANCOS                                                        1
Moral Extranjera Pais c/tratad                                1
Moral Extranj. Pais s/trata

In [9]:
df_final.value_counts("SISTEMA_ORIGEN")

SISTEMA_ORIGEN
SIRAC                           4278
TAS                             1934
MECA                             208
SIPE                             152
SIAG                             107
SIMS                              77
SOI                               22
SIRAC, TAS                        22
TAS, MECA                         18
SIPE, MECA                         9
SIPE, TAS                          7
SIPE, MECA, TAS                    5
SIRAC, SIAG                        4
SIMS, SIPE                         3
SIAG, MECA                         3
SIMS, TAS                          2
TAS, SIAG                          2
TAS, SIMS, MECA                    2
SIRAC, TAS, SIAG                   2
TAS, SIMS, MECA, SIAG              1
SIPE, SIRAC                        1
SIRAC, MECA, TAS                   1
SIPE, TAS, SIAG                    1
SIPE, SIMS, MECA                   1
SIPE, MECA, TAS, SIAG              1
SIPE, MECA, SIAG                   1
SIMS, SIAG             

In [7]:
df_final.head(10)

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
0,1,NaN,NAFIN FIDUCIARIO FIDEICOMISO CONTRAGARANTIA F...,1063586,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,5325 6914,,NFI3406305T0,NaN,NaN,GUERRERO HERRERA RAFAEL GIL,Moral Nacional no Gravable,NaN,TAS,24/02/25
1,2,NaN,".+ TASA ACTIVO, S.A. DE C.V., F.I.R.V.",1064707,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,55 5033 3341,,STA130219T22,NaN,NaN,MARQUEZ LARA RENE ALONSO,Moral Nacional no Gravable,NaN,TAS,24/02/25
2,3,NaN,".+TASA LARGO PLAZO, S.A. DE C.V., F.I.I.D.",1064327,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,55 5033 3341,,STL100514QG9,NaN,NaN,MARQUEZ LARA RENE ALONSO,Moral Nacional no Gravable,NaN,TAS,24/02/25
3,4,NaN,".+TASA PREVISIONAL, S.A. DE C.V., F.I.I.D.",1064709,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,55 5033 3341,,STR130219R32,NaN,NaN,MARQUEZ LARA RENE ALONSO,Moral Nacional no Gravable,NaN,TAS,24/02/25
4,5,NaN,".+TASA US, S.A. DE C.V., F.I.I.D.",6009236,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,55 5033 3341,,STU0606078L3,NaN,NaN,MARQUEZ LARA RENE ALONSO,Moral Nacional no Gravable,NaN,TAS,24/02/25
5,6,NaN,".+TASA, S.A. DE C.V., F.I.I.D.",1063919,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,55 50333 3341,,STC990818T34,NaN,NaN,MARQUEZ LARA RENE ALONSO,Moral Nacional no Gravable,NaN,TAS,24/02/25
6,7,NaN,3HC SUMINISTROS Y MAQUINADOS SA DE CV,28769077,VIGENTE,40620983,NO_APLICA,NaN,NaN,MEXICO,...,NaN,NaN,HSM180427T54,NaN,NaN,SANTIAGO CARRANZA GARZA,PM,NaN,SIRAC,24/02/25
7,8,NaN,4M COMERCIALIZADORA SA DE CV,28931908,VIGENTE,39976901,NO_APLICA,NaN,NaN,MEXICO,...,NaN,NaN,MCO110407D59,NaN,NaN,NaN,PM,NaN,SIRAC,24/02/25
8,9,NaN,7HOSES S DE RL DE CV.,28664082,VIGENTE,40604282,NO_APLICA,NaN,NaN,MEXICO,...,NaN,NaN,HOS1709285L6,NaN,NaN,MANUEL CELORIO GOMEZ,PM,NaN,SIRAC,24/02/25
9,10,NaN,80841 MEX-TOL CEBURES/ FONDO DE RESERVA PARA L...,1065381,VIGENTE,NaN,NaN,NaN,NaN,NaN,...,53256000,,PAC8907264G0,NaN,NaN,VAZQUEZ GASCA MARIA DEL ROCIO,Moral Nacional Gravable,NaN,TAS,24/02/25


In [5]:
df_final.describe()

,ID_DL,ENTIDAD_FEDERATIVA_NACIMIENTO,NUMERO_INTERIOR,COLONIA_URBANIZACION,FIEL,PRODUCTO_CONTRATADO
count,6868.000000,0.0,0.0,0.0,1.100000e+01,0.0
mean,3434.500000,NaN,NaN,NaN,1.000001e+15,NaN
std,1982.765157,NaN,NaN,NaN,1.505044e+08,NaN
min,1.000000,NaN,NaN,NaN,1.000000e+15,NaN
25%,1717.750000,NaN,NaN,NaN,1.000000e+15,NaN
50%,3434.500000,NaN,NaN,NaN,1.000001e+15,NaN
75%,5151.250000,NaN,NaN,NaN,1.000001e+15,NaN
max,6868.000000,NaN,NaN,NaN,1.000001e+15,NaN
